In [24]:
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
from openpyxl import load_workbook
import torch.nn as nn
from sklearn import preprocessing
import math
import pandas as pd
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score,roc_curve,auc,f1_score,balanced_accuracy_score
from sklearn import linear_model
from sklearn import svm,decomposition
#The files here is in folder /MAg/datasets/,Please note that the histogram-based training set, test set, 
#and validation set of each model will be placed in the same folder, for example, placed in /MAg/datasets/CRC_DX/resnet18/
dataset = load_workbook('D:/dataset_Huo_1/feature/resnet18/feature_train_10.xlsx',data_only=True)
dataset_x = dataset.active
dataset_test = load_workbook('D:/dataset_Huo_1/feature/resnet18/feature_test_10.xlsx',data_only=True)
dataset_tst = dataset_test.active
validation = load_workbook('D:/dataset_Huo_1/feature/resnet18/feature_val_10.xlsx',data_only=True)
val_tst = validation.active
dimension = 10 # the dimensions of the histogram
# sample of different set and class
num_train = 188 # for STAD: 124
num_val_MSS = 61 # for STAD: 49
num_val_MSI = 11 # for STAD: 12
num_test_MSS = 74 # for STAD: 74
num_test_MSI = 26 # for STAD: 25

In [25]:
# create training set, validation set, testing set
X = [[0]*10 for i in range(num_train)]
y = []
for i in range(num_train):
    for j in range(10):
        X[i][j] = dataset_x.cell(i+1,j+1).value
    y.append(float(dataset_x.cell(i+1,12).value))

X_val = [[0]*dimension for i in range(num_val_MSS+num_val_MSI)]
y_val = []
for i in range(num_val_MSS+num_val_MSI):
    for j in range(10):
        X_val[i][j] = val_tst.cell(i+1,j+1).value 
    y_val.append(float(val_tst.cell(i+1,12).value))

X_test = [[0]*dimension for i in range(num_test_MSS+num_test_MSI)]
y_test = []
for i in range(num_test_MSS+num_test_MSI):
    for j in range(10):
        X_test[i][j] = dataset_tst.cell(i+1,j+1).value 
    y_test.append(float(dataset_tst.cell(i+1,12).value))

# make the predicting label list for validation set and testing set
true_val_label = []
for j in range(num_val_MSS):
    true_val_label.append(0)
for i in range(num_val_MSI):
    true_val_label.append(1)
    
true_label = []
for j in range(num_test_MSS):
    true_label.append(0)
for i in range(num_test_MSI):
    true_label.append(1)

In [26]:
clt = svm.SVC(C = 1,kernel = 'sigmoid',probability = True,class_weight = {0:0.0595,1:0.9405})
clt.fit(X,y)
test = [[]]

'''
validation
'''
pred_label_val = clt.predict(X_val)
# print(label)
right_mss = 0
right_msi = 0
for k in range(num_val_MSS):
    if pred_label_val[k] == 0:
        right_mss = right_mss + 1
for k in range(num_val_MSI):
    if pred_label_val[k + num_val_MSS] == 1:
        right_msi = right_msi + 1
print('right mss:',right_mss,'/',num_val_MSS)
print('right msi:',right_msi,'/',num_val_MSI)
print('f1_val:',f1_score(y_val,pred_label_val))
print('BACC:',balanced_accuracy_score(y_val,pred_label_val))
print('acc:', right_mss + right_msi,'/',num_val_MSI+num_val_MSS)
print('acc:', (right_mss + right_msi) / (num_val_MSI+num_val_MSS))

right mss: 45 / 61
right msi: 6 / 11
f1_val: 0.3636363636363636
BACC: 0.6415797317436662
acc: 51 / 72
acc: 0.7083333333333334


In [27]:
'''
test the trained SVM model
'''
pred_label_test = clt.predict(X_test)
right_mss = 0
right_msi = 0
for k in range(num_test_MSS):
    if pred_label_test[k] == 0:
        right_mss = right_mss + 1
for k in range(num_test_MSI):
    if pred_label_test[k + num_test_MSS] == 1:
        right_msi = right_msi + 1
print('right mss:',right_mss,'/',num_test_MSS)
print('right msi:',right_msi,'/',num_test_MSI)
print('f1:',f1_score(true_label,pred_label_test))
print('BACC:',balanced_accuracy_score(true_label,pred_label_test))
print('acc:', right_mss + right_msi,'/',(num_test_MSI+num_test_MSS))
print('acc:', (right_mss + right_msi) / (num_test_MSI+num_test_MSS))

right mss: 58 / 74
right msi: 19 / 26
f1: 0.6229508196721311
BACC: 0.7572765072765073
acc: 77 / 100
acc: 0.77
